## Diagnostics

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.stats as ss

import elfi

import logging
logging.basicConfig(level=logging.INFO)

seed = 37

### Summary-statistics Selection

In [2]:
from elfi.methods.diagnostics import TwoStageSelection

ELFI implements the Two Stage Procedure proposed by Nunes & Balding (2010). The intention to run the Two Stage Procedure is to determine a well-performing summary-statistics combination. Briefly, the procedure's rationale can be summarised as follows:
- First, the module computes or accepts the combinations of the candidate summary statistics;
- In Stage 1, each summary-statistics combination is evaluated using the
Minimum Entropy algorithm;
- In Stage 2, the minimum-entropy combination is selected,
and the 'closest' datasets are identified;
- Further in Stage 2, for each summary-statistics combination,
the mean root sum of squared errors (MRSSE) is calculated over all 'closest datasets',
and the minimum-MRSSE combination is chosen as the one with the optimal performance.

#### Example 1: Summary-statistics selection for a 1-D Gaussian-noise model.

For the first example application, we define a Gaussian-noise model and three summary statistics, one of which is uninformative.

In [3]:
def fn_gaussian_simulator(mu, sigma, batch_size=1, random_state=None):
    mu, sigma = np.atleast_1d(mu, sigma)
    res = ss.norm.rvs(mu[:, None], sigma[:, None], size=(batch_size, 5), random_state=random_state)
    return res

def mean(y):
    return np.mean(y, axis=1)

def var(y):
    return np.var(y, axis=1)

def uninformative(y):
    return 1

To carry the Two Stage procedure, we assess a list of summary statistics based on some initialised simulator.

In [4]:
# Obtaining the observations.
mean_obs = 1
std_obs = 3
y_obs = fn_gaussian_simulator(mean_obs, std_obs, random_state=np.random.RandomState(seed))

# Initialising the priors.
prior_mu = elfi.Prior('uniform', -2, 4, name='mu')
prior_sigma = elfi.Prior('uniform', 1, 4, name='sigma')

# Initialising the simulator.
simulator_gaussian = elfi.Simulator(fn_gaussian_simulator, prior_mu, prior_sigma, observed=y_obs)

# Initialising the list of assessed summary statistics.
list_ss = [mean, var, uninformative]

Now, we are ready to initialise the Two Stage Selection. Note that we can choose a distance metric based on which the summary statistics will be evaluated.

In [5]:
diagnostics = TwoStageSelection(simulator_gaussian, 'euclidean', list_ss=list_ss, seed=seed)

In this Two Stage procedure's implementation, we can choose the number of simulations `n_sim` obtained for every assessed summary-statistics combination and the number of such accepted simulations `n_acc`.

Note that the rationale of the Two Stage procedure is based on finding the combination with the minimum entropy (Stage 1) and then finding the collection which shows the minimum mean sum of squared error (MRRSE) based on the parameters corresponding to the `n_closest` datasets of the minimum-entropy combination (Stage 2).

In [6]:
diagnostics.run(n_sim=1000, n_acc=100, n_closest=20)

INFO:elfi.methods.diagnostics:Combination ['mean'] shows the entropy of 2.276587
INFO:elfi.methods.diagnostics:Combination ['var'] shows the entropy of 2.331251
INFO:elfi.methods.diagnostics:Combination ['uninformative'] shows the entropy of 2.390263
INFO:elfi.methods.diagnostics:Combination ['mean', 'var'] shows the entropy of 2.183706
INFO:elfi.methods.diagnostics:Combination ['mean', 'uninformative'] shows the entropy of 2.276587
INFO:elfi.methods.diagnostics:Combination ['var', 'uninformative'] shows the entropy of 2.331251
INFO:elfi.methods.diagnostics:Combination ['mean', 'var', 'uninformative'] shows the entropy of 2.183706
INFO:elfi.methods.diagnostics:
The minimum entropy of 2.183706 was found in ['mean', 'var'].

INFO:elfi.methods.diagnostics:Combination ['mean'] shows the MRSSE of 17.146237
INFO:elfi.methods.diagnostics:Combination ['var'] shows the MRSSE of 18.754292
INFO:elfi.methods.diagnostics:Combination ['uninformative'] shows the MRSSE of 18.906968
INFO:elfi.methods.d

(<function __main__.mean>, <function __main__.var>)

#### Example 2: Summary-statistics selection for the MA2 model.

The second example is ran on the MA2 example model, which is introduced in `tutorial.ipynb`. We evaluate two autocovariance summary statistics: one with `lag=1`, another with `lag=3`, and the mean summary statistic. The preparation for the diagnostics given below is identical to the one performed in Example 1.

In [7]:
# Defining the simulator.
def fn_MA2_simulator(t1, t2, n_obs=100, batch_size=1, random_state=None):
    # Make inputs 2d arrays for numpy broadcasting with w
    t1 = np.asanyarray(t1).reshape((-1, 1))
    t2 = np.asanyarray(t2).reshape((-1, 1))
    random_state = random_state or np.random

    w = random_state.randn(batch_size, n_obs+2)  # i.i.d. sequence ~ N(0,1)
    x = w[:, 2:] + t1*w[:, 1:-1] + t2*w[:, :-2]
    return x

# Defining the priors.
class CustomPrior_t1(elfi.Distribution):
    def rvs(b, size=1, random_state=None):
        u = ss.uniform.rvs(loc=0, scale=1, size=size, random_state=random_state)
        t1 = np.where(u<0.5, np.sqrt(2.*u)*b-b, -np.sqrt(2.*(1.-u))*b+b)
        return t1
class CustomPrior_t2(elfi.Distribution):
    def rvs(t1, a, size=1, random_state=None):
        locs = np.maximum(-a-t1, t1-a)
        scales = a - locs
        t2 = ss.uniform.rvs(loc=locs, scale=scales, size=size, random_state=random_state)
        return t2
    
# Defining the assessed summary statistics.
def autocov(y, lag=1):
    C = np.mean(y[:,lag:] * y[:,:-lag], axis=1)
    return C
def autocov_3(y, lag=3):
    C = np.mean(y[:,lag:] * y[:,:-lag], axis=1)
    return C
def mean(y):
    return np.mean(y, axis=1)

# Obtaining the observations.
t1_obs = 1
t2_obs = 3
y_obs = fn_MA2_simulator(t1_obs, t2_obs, random_state=np.random.RandomState(seed))

# Initialising the priors.
prior_t1 = elfi.Prior(CustomPrior_t1, 2, name='prior_t1')
prior_t2 = elfi.Prior(CustomPrior_t2, prior_t1, 1, name='prior_t2')

# Initialising the simulator.
simulator_MA2 = elfi.Simulator(fn_MA2_simulator, prior_t1, prior_t2, observed=y_obs)

# Initialising the list of assessed summary statistics.
list_ss = [autocov, autocov_3, mean]

After the preparation, we can executing the diagnostics as follows:

In [8]:
diagnostics = TwoStageSelection(simulator_MA2, 'euclidean', list_ss=list_ss, seed=seed)
diagnostics.run(n_sim=1000, n_acc=100, n_closest=20)

INFO:elfi.methods.diagnostics:Combination ['autocov'] shows the entropy of 2.844670
INFO:elfi.methods.diagnostics:Combination ['autocov_3'] shows the entropy of 4.476933
INFO:elfi.methods.diagnostics:Combination ['mean'] shows the entropy of 3.927929
INFO:elfi.methods.diagnostics:Combination ['autocov', 'autocov_3'] shows the entropy of 2.938405
INFO:elfi.methods.diagnostics:Combination ['autocov', 'mean'] shows the entropy of 2.804756
INFO:elfi.methods.diagnostics:Combination ['autocov_3', 'mean'] shows the entropy of 4.423709
INFO:elfi.methods.diagnostics:Combination ['autocov', 'autocov_3', 'mean'] shows the entropy of 2.948902
INFO:elfi.methods.diagnostics:
The minimum entropy of 2.804756 was found in ['autocov', 'mean'].

INFO:elfi.methods.diagnostics:Combination ['autocov'] shows the MRSSE of 22.686602
INFO:elfi.methods.diagnostics:Combination ['autocov_3'] shows the MRSSE of 26.990657
INFO:elfi.methods.diagnostics:Combination ['mean'] shows the MRSSE of 23.081421
INFO:elfi.metho

(<function __main__.autocov>,
 <function __main__.autocov_3>,
 <function __main__.mean>)